In [48]:
# %pip install --no-cache-dir --force-reinstall numpy==1.26.4 pandas==2.2.2

In [49]:
# %pip install nba_api --quiet

In [3]:
!pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl (12.9 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install tqdm  nba_api

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/284.9 kB ? eta -:--:--
   -- ------------------------------------ 20.5/284.9 kB 640.0 kB/s eta 0:00:01
   ------------ --------------------------- 92.2/284.9 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- - 276.5/284.9 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 284.9/284.9 kB 2.2 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp311-cp311-win_amd64.whl (102 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-an


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import time
from tqdm import tqdm
import numpy as np
from collections import defaultdict
from nba_api.stats.endpoints import (PlayByPlayV2,
                                     BoxScoreSummaryV2,
                                     BoxScoreTraditionalV2,
                                     LeagueGameFinder)

In [2]:
gamefinder = LeagueGameFinder(season_nullable='2023-24', season_type_nullable='Regular Season')
games_df = gamefinder.get_data_frames()[0]
all_game_ids = games_df['GAME_ID'].unique().tolist()

In [3]:
# Helper to convert clock time string to seconds
def time_str_to_sec(t):
    if isinstance(t, str):
        m, s = map(int, t.strip().split(":"))
        return m * 60 + s
    return 0

# Helper to compute duration between lineup segments
def compute_duration(row):
    start_sec = time_str_to_sec(row['start_time'])
    end_sec = time_str_to_sec(row['end_time'])
    start_period = row['period']
    end_period = row['period']
    if end_sec > start_sec:
        end_period = start_period + 1
    period_diff = end_period - start_period
    return start_sec + period_diff * 720 - end_sec

In [4]:
game_ids = all_game_ids[0:50] # next is [50:100]

In [5]:
previous_lineup_vs_lineup_df = pd.read_csv('lineup_vs_lineup.csv', dtype={'game_id': 'object'})
previous_player_stats_df = pd.read_csv('player_stats.csv', dtype={'game_id': 'object'})

In [6]:
# All missing lineup game_ids

all_game_ids_series = pd.Series(all_game_ids)

missing_game_ids = all_game_ids_series[~all_game_ids_series.isin(previous_lineup_vs_lineup_df['game_id'].unique())].tolist()
print(f"Missing game_ids: {missing_game_ids}")

Missing game_ids: ['0022300197', '0022300192', '0022300195', '0022300198', '0022300196', '0022300199', '0022300194', '0022300193', '0022300024', '0022300017', '0022300018', '0022300019', '0022300021', '0022300023', '0022300026', '0022300025', '0022300020', '0022300022', '0022300189', '0022300190', '0022300188', '0022300191', '0022300181', '0022300183', '0022300179', '0022300177', '0022300185', '0022300178', '0022300186', '0022300180', '0022300184', '0022300182', '0022300187', '0022300173', '0022300174', '0022300175', '0022300176', '0022300011', '0022300009', '0022300010', '0022300012', '0022300013', '0022300016', '0022300014', '0022300015', '0022300008', '0022300171', '0022300172', '0022300163', '0022300157', '0022300168', '0022300170', '0022300160', '0022300158', '0022300161', '0022300159', '0022300164', '0022300165', '0022300166', '0022300169', '0022300167', '0022300162', '0022300155', '0022300146', '0022300145', '0022300156', '0022300149', '0022300153', '0022300148', '0022300154', '

In [7]:
# Store all results
all_lineup_vs_lineup = []
all_player_stats = []
nr = 1
for game_id in tqdm(missing_game_ids):
    try:
        # --- Load game data ---
        print(f"----------------GAME nr.{nr}-----------\n")
        nr += 1

        pbp = PlayByPlayV2(game_id=game_id).get_data_frames()[0]
        box = BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]
        summary = BoxScoreSummaryV2(game_id=game_id).get_data_frames()[0]

        home_team_id = int(summary['HOME_TEAM_ID'][0])
        away_team_id = int(summary['VISITOR_TEAM_ID'][0])

        starters_df = box[box['START_POSITION'] != '']
        starters_by_team = starters_df.groupby('TEAM_ID')['PLAYER_NAME'].apply(list).to_dict()

        current_lineups = {
            home_team_id: starters_by_team.get(home_team_id, [])[:],
            away_team_id: starters_by_team.get(away_team_id, [])[:]
        }

        # --- Build lineup_df ---
        important_event_types = list(range(1, 14))
        lineup_timeline = []

        for _, row in pbp.iterrows():
            event_type = row['EVENTMSGTYPE']
            home_desc = str(row['HOMEDESCRIPTION']) if pd.notna(row['HOMEDESCRIPTION']) else ''
            away_desc = str(row['VISITORDESCRIPTION']) if pd.notna(row['VISITORDESCRIPTION']) else ''

            if event_type == 8:
                out_player = row['PLAYER1_NAME']
                in_player = row['PLAYER2_NAME']
                team_id = row['PLAYER1_TEAM_ID'] if pd.notna(row['PLAYER1_TEAM_ID']) else row['PLAYER2_TEAM_ID']
                team_id = int(team_id) if pd.notna(team_id) else None

                if team_id in current_lineups:
                    if out_player in current_lineups[team_id]:
                        current_lineups[team_id].remove(out_player)
                    if in_player not in current_lineups[team_id]:
                        current_lineups[team_id].append(in_player)

            if event_type in important_event_types:
                lineup_timeline.append({
                    'EVENTNUM': row['EVENTNUM'],
                    'PERIOD': row['PERIOD'],
                    'TIME': row['PCTIMESTRING'],
                    'EVENT_TYPE': event_type,
                    'SCORE': row['SCORE'],
                    'HOME_DESCRIPTION': home_desc,
                    'AWAY_DESCRIPTION': away_desc,
                    'HOME_LINEUP': current_lineups[home_team_id][:],
                    'AWAY_LINEUP': current_lineups[away_team_id][:],
                    'PLAYER1_NAME': row['PLAYER1_NAME'],
                    'PLAYER2_NAME': row['PLAYER2_NAME'],
                    'PLAYER3_NAME': row['PLAYER3_NAME']
                })

        lineup_df = pd.DataFrame(lineup_timeline)

        # --- Build lineup segments ---
        lineup_segments = []
        segment = None
        prev_home = None
        prev_away = None
        last_shot_team = None
        SKIP_KEYWORDS = ['SUB', 'Jump Ball', 'Delay', 'Offensive', 'Timeout', 'Rebound']

        for _, row in lineup_df.iterrows():
            home_lineup = tuple(sorted(row['HOME_LINEUP']))
            away_lineup = tuple(sorted(row['AWAY_LINEUP']))

            if segment is None or home_lineup != prev_home or away_lineup != prev_away:
                if segment:
                    segment['end_event'] = row['EVENTNUM']
                    segment['end_time'] = row['TIME']
                    lineup_segments.append(segment)
                segment = {
                    'home_lineup': home_lineup,
                    'away_lineup': away_lineup,
                    'start_event': row['EVENTNUM'],
                    'start_time': row['TIME'],
                    'period': row['PERIOD'],
                    'team_stats': {'home': defaultdict(int), 'away': defaultdict(int)},
                    'player_stats': defaultdict(lambda: defaultdict(int)),
                    'events': []
                }
                prev_home = home_lineup
                prev_away = away_lineup

            segment['events'].append({
                'EVENTNUM': row['EVENTNUM'],
                'TIME': row['TIME'],
                'HOME_DESCRIPTION': row['HOME_DESCRIPTION'],
                'AWAY_DESCRIPTION': row['AWAY_DESCRIPTION']
            })

            for team_side, desc in [('home', row['HOME_DESCRIPTION']), ('away', row['AWAY_DESCRIPTION'])]:
                if not desc or any(skip_word in desc for skip_word in SKIP_KEYWORDS):
                    continue

                lineup = segment[f'{team_side}_lineup']
                players_on_court = set(lineup)

                def inc(stat, players, value=1):
                    segment['team_stats'][team_side][stat] += value
                    for p in players:
                        if p in players_on_court:
                            segment['player_stats'][p][stat] += value

                players = [p for p in [row['PLAYER1_NAME'], row['PLAYER2_NAME'], row['PLAYER3_NAME']]
                           if isinstance(p, str) and p in players_on_court]

                if '3PT' in desc and 'MISS' not in desc:
                    inc('3pt_made', [row['PLAYER1_NAME']])
                    inc('points', [row['PLAYER1_NAME']], value=3)
                elif 'Free Throw' in desc and 'MISS' not in desc:
                    inc('ft_made', [row['PLAYER1_NAME']])
                    inc('points', [row['PLAYER1_NAME']], value=1)
                elif 'MISS' not in desc and any(kw in desc for kw in ['Fadeaway', 'Dunk', 'Layup', 'Jump Shot', 'Hook Shot']):
                    inc('2pt_made', [row['PLAYER1_NAME']])
                    inc('points', [row['PLAYER1_NAME']], value=2)
                elif 'MISS' in desc:
                    last_shot_team = team_side

                if 'AST' in desc: inc('assists', [row['PLAYER2_NAME']])
                if 'Turnover' in desc: inc('turnovers', [row['PLAYER1_NAME']])
                if 'STL' in desc: inc('steals', [row['PLAYER2_NAME']])
                if 'BLK' in desc: inc('blocks', [row['PLAYER2_NAME']])
                if '.FOUL' in desc: inc('fouls', [row['PLAYER1_NAME']])

                if 'REBOUND' in desc:
                    rebounder = row['PLAYER1_NAME']
                    if last_shot_team:
                        if team_side == last_shot_team:
                            inc('off_rebounds', [rebounder])
                        else:
                            inc('def_rebounds', [rebounder])
                        last_shot_team = None

        if segment:
            segment['end_event'] = row['EVENTNUM']
            segment['end_time'] = row['TIME']
            lineup_segments.append(segment)

        # --- Build lineup_vs_lineup_df ---
        for seg in lineup_segments:
            row = {
                'game_id': game_id,
                'period': seg['period'],
                'start_event': seg['start_event'],
                'end_event': seg['end_event'],
                'start_time': seg['start_time'],
                'end_time': seg['end_time'],
                'home_lineup': seg['home_lineup'],
                'away_lineup': seg['away_lineup'],
            }
            for team in ['home', 'away']:
                for stat, value in seg['team_stats'][team].items():
                    row[f'{team}_{stat}'] = value
            row['duration'] = compute_duration(row)
            all_lineup_vs_lineup.append(row)

        # --- Build player_stats_df ---
        for seg in lineup_segments:
            for player_name, stats in seg['player_stats'].items():
                row = {
                    'game_id': game_id,
                    'player': player_name,
                    'period': seg['period'],
                    'start_event': seg['start_event'],
                    'end_event': seg['end_event'],
                    'start_time': seg['start_time'],
                    'end_time': seg['end_time'],
                    'home_lineup': seg['home_lineup'],
                    'away_lineup': seg['away_lineup'],
                }
                for stat, value in stats.items():
                    row[stat] = value
                all_player_stats.append(row)

        print(f"Finished game: {game_id}")
        
        # Convert to DataFrame
        lineup_vs_lineup_df = pd.DataFrame(all_lineup_vs_lineup)
        player_stats_df = pd.DataFrame(all_player_stats)

        # Concatenate with previous data
        lineup_vs_lineup_df = pd.concat([previous_lineup_vs_lineup_df, lineup_vs_lineup_df], ignore_index=True)
        player_stats_df = pd.concat([previous_player_stats_df, player_stats_df], ignore_index=True)

        # Save to CSV
        lineup_vs_lineup_df.to_csv('lineup_vs_lineup.csv', index=False)
        player_stats_df.to_csv('player_stats.csv', index=False)
        print("Data saved to CSV files.")

        # Respect rate limits
        time.sleep(1)

    except Exception as e:
        print(f"Failed on game {game_id}: {e}")
        continue

  0%|          | 0/165 [00:00<?, ?it/s]

----------------GAME nr.1-----------

Finished game: 0022300197
Data saved to CSV files.


  1%|          | 1/165 [00:10<29:59, 10.97s/it]

----------------GAME nr.2-----------

Finished game: 0022300192
Data saved to CSV files.


  1%|          | 2/165 [00:16<21:50,  8.04s/it]

...

 99%|█████████▉| 163/165 [19:38<00:12,  6.37s/it]

----------------GAME nr.164-----------

Finished game: 0022300062
Data saved to CSV files.


 99%|█████████▉| 164/165 [19:45<00:06,  6.52s/it]

----------------GAME nr.165-----------

Finished game: 0022300061
Data saved to CSV files.


100%|██████████| 165/165 [19:50<00:00,  7.22s/it]
